# Hyperparameter Tuning using HyperDrive

Importing dependencies required for the project.

In [ ]:
from azureml.core import Dataset, Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

import os
import shutil
import joblib

## Dataset

### Overview

The Dataset used is "Stroke prediction Dataset"

According to the World Health Organization (WHO) stroke is the 2nd leading cause of death globally, responsible for approximately 11% of total deaths.
This dataset is used to predict whether a patient is likely to get stroke based on the input parameters like gender, age, various diseases, and smoking status. Each row in the data provides relavant information about the patient.

<b>Attribute Information </b>

1) id: unique identifier

2) gender: "Male", "Female" or "Other"

3) age: age of the patient

4) hypertension: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension

5) heart_disease: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease

6) ever_married: "No" or "Yes"

7) work_type: "children", "Govt_jov", "Never_worked", "Private" or "Self-employed"

8) Residence_type: "Rural" or "Urban"

9) avg_glucose_level: average glucose level in blood

10) bmi: body mass index

11) smoking_status: "formerly smoked", "never smoked", "smokes" or "Unknown"*

12) stroke: 1 if the patient had a stroke or 0 if not

In this project, Azure AutoML will be used to make prediction on the death event based on patient's 10clinical features.

In [ ]:
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="hd-experiment")
dataset = Dataset.get_by_name(ws, name='stroke-predicton')

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

## Configure Compute Cluster 

In [ ]:
cpu_cluster = "cpu-cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D12_V2',
                                                           max_nodes=5)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

## Hyperdrive Configuration

Hyperparamters can be discrete or continuous. Discrete parameters are specified as a choice among dicrete values. Continuous hyperparameters are specified as a distribution over a range of values. I chose 'choice' for describing the values of -C and -max_iter. The sampling method I used in RandomSampling. It supports both discrete and continuous values. It supports early termination of low-performance runs. In Random Sampling, the values are selected randomly from a defined search space.

With the help of Early Termination policy, we can terminate poorly performing runs. The policy that I chose is BanditPolicy. Bandit Policy is based on slack factor/slack amount and evaluation interval. This policy will terminate runs whose primary metric is not within the specified slack factor/slack amount. By using this policy we could improve the computational efficiency.

For the Hyperdrive Configuration the Primary metric chosen is Accuracy, Max concurrent run is 4 and Max total run is set as 100.

In [ ]:
ps = RandomParameterSampling({
        "--C": uniform(0.1,1.0),
        "--max_iter": choice(50, 100, 150, 200)
    })

# Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 3, delay_evaluation = 3)

# SKLearn estimator for use with train.py
est = SKLearn(source_directory = '.', 
              compute_target = cpu_cluster, 
              entry_script = 'train.py')

# HyperDriveConfig using hyperparameter sampler, policy and estimator.
hyperdrive_config = HyperDriveConfig(estimator = est,
                                hyperparameter_sampling = ps,
                                policy = policy,
                                primary_metric_name = 'Accuracy',
                                primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs = 100,
                                max_concurrent_runs = 4)

In [ ]:
# Submitting the hyperdrive run to the experiment
hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)

## Run Details



In [ ]:

RunDetails(hyperdrive_run).show()
hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run ID: ', best_run.id)
print('\n Metrics: ', best_run_metrics)
print('\n Parameters: ', parameter_values)
print('\n Accuracy: ', best_run_metrics['Accuracy'])
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run ID: ', best_run.id)
print('\n Metrics: ', best_run_metrics)
print('\n Parameters: ', parameter_values)
print('\n Accuracy: ', best_run_metrics['Accuracy'])


In [ ]:
best_run

In [ ]:
best_run.download_file('/outputs/model.joblib', 'hyperdrive_model.joblib')

In [ ]:
#Register the best model
best_model_registered = best_run.register_model(model_name = 'best_hyperdrive_model', 
                                                model_path='outputs/model.joblib',
                                                description='best hyperdrive model')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service